In [1]:
!pip install -f https://download.pytorch.org/whl/cu101/torch_stable.html torch==1.8.0+cu101 torchvision==0.9.0
!pip install wandb==0.12.1

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html


In [47]:
from tqdm import tqdm
from os import listdir
from random import *
seed(0)


import pandas as pd

from pylab import *

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import wandb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "сpu"

### Data trepanation

In [48]:
dataset = 'barents'
folder_train, folder_val, folder_test = dataset + '/train/maps/', dataset + '/valid/maps/', dataset + '/test/maps/'
train_files, val_files, test_files = sorted([file for file in listdir(folder_train)])[1346:],\
                                     sorted([file for file in listdir(folder_val)]),\
                                     sorted([file for file in listdir(folder_test)]) # + 1200

sample = torch.load(folder_train + train_files[1])
grid = torch.load(dataset + "/train/grid.pt")
coverage = pd.read_csv(dataset + "/train/coverage.csv")

len(train_files), len(val_files), len(test_files)

(1812, 366, 365)

In [49]:
def preprocess_image(tensor: torch.Tensor) -> torch.Tensor:
    return (torch.nan_to_num(tensor, nan=-10.0) + grid['land']*10)

In [50]:
images_train = [preprocess_image(torch.load(folder_train + train_files[i])["jaxa.sic"])\
                            for i in tqdm(range(0, len(train_files)))]

images_val = [preprocess_image(torch.load(folder_val + val_files[i])["jaxa.sic"])\
                            for i in tqdm(range(0, len(val_files)))]

def handler(data: dict):
    try:
        return data["jaxa.sic"]
    except:
        return data["jaxa.sic_asc"]
    
images_test = [preprocess_image(handler(torch.load(folder_test + test_files[i])))\
                            for i in tqdm(range(0, len(test_files)))]

images_tensor_train, images_tensor_val, images_tensor_test = torch.stack(images_train),\
                                                             torch.stack(images_val),\
                                                             torch.stack(images_test)
images_tensor_train.shape

 52%|█████▏    | 189/366 [00:01<00:01, 135.27it/s]

KeyboardInterrupt: 

In [40]:
def average(data: list) -> float:
    return sum(data)/len(data)

def load_data(batch_size, images_tensor, d_in, d_out):
    in_batch, out_batch = [], []
    for i in range(batch_size):
        start_point = choice(range(len(images_tensor) - d_in - d_out))
        in_batch.append(images_tensor[start_point:start_point+d_in])
        out_batch.append(images_tensor[start_point+d_in:start_point+d_in+d_out])
    return torch.stack(in_batch), torch.stack(out_batch)

def train(model, criterion, d_in, d_out, epochs, batch_size, lr, eval_step):
    mae_total, rmse_total, mape_total = [], [], []

    total_test_len = len(images_tensor_test)
    start_point, steps = 0, int(total_test_len/(d_in + d_out))

    for i in range(steps):
        model_out, images_out = torch.stack([images_tensor_test[start_point+d_in] for i in range(d_out)]),\
                                images_tensor_test[start_point+d_in+1:start_point+d_in+d_out+1]

        start_point += d_in + d_out


        criterion_mse = nn.MSELoss()

        loss_mse = criterion_mse(model_out, images_out[None, :, :, :])
        loss_mae = criterion(model_out, images_out[None, :, :, :])

        mae_total.append(loss_mae.detach().cpu().numpy())
        rmse_total.append(math.sqrt(loss_mse.detach().cpu().numpy()))


    wandb.log(
        {
         "test/test_MAE": average(mae_total), 
         "test/test_RMSE": average(rmse_total)
        }
    )

    print('Epoch ', 0, ', test MAE - ', average(mae_total))

In [14]:
architecture = "Persistance_7:1"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-07-26 13:09:35.731820: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [46]:
epochs = 1500
d_in, d_out = 1,10 
batch_size = 8
lr = 1e-3
eval_step = 20
criterion = nn.L1Loss()
model_a = []

train(model_a, criterion, d_in, d_out, epochs, batch_size, lr, eval_step)

Epoch  0 , test MAE -  2.496330492424242


In [7]:
from torchvision.models import resnet50

In [8]:
from torchvision import datasets, transforms, models

In [9]:
model = models.resnet152(pretrained=True)
#model = resnet50()

In [15]:
model.conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model = torch.nn.Sequential(*(list(model.children())[:-2])).to(device)

In [13]:
test_input = torch.zeros(4, 7, 360, 500).to(device)

In [22]:
conv_int1 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1, bias=False).to(device)
conv_int2 = nn.Conv2d(512, 511, kernel_size=3, padding=1, bias=False).to(device)

up1 = nn.ConvTranspose2d(2048, 2048 // 2, kernel_size=2, stride=(2,2)).to(device)
up2 = nn.ConvTranspose2d(1024, 1024 // 2, kernel_size=3, stride=(3,3)).to(device)
up3 = nn.ConvTranspose2d(511, 511 // 170, kernel_size=(5, 25), stride=(5,5)).to(device)
#up4 = nn.ConvTranspose2d(3, 3, kernel_size=(1,21), stride=(1,1)).to(device)

#print(resnet152_aug(test_input).shape)
print(conv_int1(up1(model(test_input))).shape)
print(up2(up1(model(test_input))).shape)
print(conv_int2(up2(up1(model(test_input)))).shape)
print(up3(conv_int2(up2(up1(model(test_input))))).shape)
#print(up4(up3(conv_int2(up2(up1(model(test_input)))))).shape)
#print(up5(up4(up3(up2(up1(model(test_input)))))).shape)
#print(up6(up5(up4(up3(up2(up1(model(test_input))))))).shape)

torch.Size([4, 1024, 24, 32])
torch.Size([4, 512, 72, 96])
torch.Size([4, 511, 72, 96])
torch.Size([4, 3, 360, 500])


In [10]:
model.requires_grad = False

In [16]:
resnet152_aug = nn.Sequential(
    model,
    nn.ConvTranspose2d(2048, 2048 // 2, kernel_size=2, stride=(2,2)),
    nn.Conv2d(1024, 1024, kernel_size=3, padding=1, bias=False),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(1024, 1024 // 2, kernel_size=3, stride=(3,3)),
    nn.Conv2d(512, 511, kernel_size=3, padding=1, bias=False),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(511, 511 // 511, kernel_size=(5, 25), stride=(5,5)).to(device)
).to(device)

In [17]:
resnet152_aug(test_input).shape

torch.Size([4, 1, 360, 500])

In [ ]:
from models.unet.unet.unet_model import UNet

epochs = 1500
d_in, d_out = 7, 1
batch_size = 8
lr = 1e-3
eval_step = 20
criterion = nn.L1Loss()
model_a = resnet152_aug.to(device)

train(resnet152_aug, criterion, d_in, d_out, epochs, batch_size, lr, eval_step)

Epoch  20 , test MAE -  9.794620090060764
Epoch  40 , test MAE -  9.406392415364584
Epoch  60 , test MAE -  8.692482503255208
Epoch  80 , test MAE -  8.557862006293403
Epoch  100 , test MAE -  8.664781358506945
Epoch  120 , test MAE -  8.152642822265625
Epoch  140 , test MAE -  8.534897189670138
Epoch  160 , test MAE -  7.208463541666666
Epoch  180 , test MAE -  6.957613796657986
Epoch  200 , test MAE -  6.357668050130209
Epoch  220 , test MAE -  5.774461534288195
Epoch  240 , test MAE -  6.263149007161458
Epoch  260 , test MAE -  6.740200466579861
Epoch  280 , test MAE -  5.680073377821181
Epoch  300 , test MAE -  5.202086724175347
Epoch  320 , test MAE -  5.172831556532118
Epoch  340 , test MAE -  5.284344482421875
Epoch  360 , test MAE -  5.218190171983507
Epoch  380 , test MAE -  4.885006374782986
Epoch  400 , test MAE -  5.458393351236979
Epoch  420 , test MAE -  4.994831339518229
Epoch  440 , test MAE -  4.921678331163195
Epoch  460 , test MAE -  5.898794216579861
Epoch  480 , te

In [ ]:
train(resnet152_aug, criterion, d_in, d_out, 10000, 16, 1e-4, eval_step)